# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
!conda install -c conda-forge gmaps

^C


In [4]:
!jupyter nbextension enable --py --sys-prefix gmaps
# OR jupyter nbextension enable --py gmaps
# install Jupyter Widgets using conda


Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [5]:
!conda install -c conda-forge ipywidgets


^C


In [6]:
# enable jupyter extensions
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
cities_df = pd.read_csv('cities.csv')
cities_df.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Krasnosel'kup,65.7000,82.4667,-14.33,94,100,5.64,RU,1618620559
1,1,Vaini,-21.2000,-175.2000,27.00,65,20,2.06,TO,1618620516
2,2,Verkhoyansk,67.5447,133.3850,-3.33,74,100,5.94,RU,1618620560
3,3,Port Elizabeth,-33.9180,25.5701,16.00,72,0,6.69,ZA,1618620560
4,4,Saskylakh,71.9167,114.0833,-15.37,94,100,6.66,RU,1618620560


In [9]:
del cities_df['Unnamed: 0']
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Krasnosel'kup,65.7000,82.4667,-14.33,94,100,5.64,RU,1618620559
1,Vaini,-21.2000,-175.2000,27.00,65,20,2.06,TO,1618620516
2,Verkhoyansk,67.5447,133.3850,-3.33,74,100,5.94,RU,1618620560
3,Port Elizabeth,-33.9180,25.5701,16.00,72,0,6.69,ZA,1618620560
4,Saskylakh,71.9167,114.0833,-15.37,94,100,6.66,RU,1618620560


In [23]:
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Krasnosel'kup,65.7000,82.4667,-14.33,94,100,5.64,RU,1618620559
1,Vaini,-21.2000,-175.2000,27.00,65,20,2.06,TO,1618620516
2,Verkhoyansk,67.5447,133.3850,-3.33,74,100,5.94,RU,1618620560
3,Port Elizabeth,-33.9180,25.5701,16.00,72,0,6.69,ZA,1618620560
4,Saskylakh,71.9167,114.0833,-15.37,94,100,6.66,RU,1618620560
...,...,...,...,...,...,...,...,...,...
543,Nhamundá,-2.1861,-56.7131,23.74,97,100,1.29,BR,1618620727
544,Vanavara,60.3400,102.2797,-11.92,68,79,5.72,RU,1618620794
545,Akdepe,42.0551,59.3788,14.00,47,0,4.12,TM,1618620794
546,Finschhafen,-6.6024,147.8681,28.87,71,90,1.92,PG,1618620795


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [11]:
locations = cities_df[['Lat','Lng']]
weight = cities_df['Humidity']

In [15]:
# Plot Heatmap
fig = gmaps.figure(center=(20,0),zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=350,
                                 point_radius=5)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

#save figure


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [37]:

ideal_df = cities_df.loc[(cities_df['Max Temp']<= 80) & (cities_df['Max Temp']> 70) & (cities_df['Wind Speed']< 10) | (cities_df['Cloudiness'] ==0)]
ideal_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,Port Elizabeth,-33.9180,25.5701,16.00,72,0,6.69,ZA,1618620560
18,Chatrapur,19.3500,84.9833,26.64,73,0,3.39,IN,1618620565
24,Xianshuigu,38.9850,117.3828,13.00,19,0,7.00,CN,1618620568
27,Cape Town,-33.9258,18.4232,18.33,77,0,0.51,ZA,1618620395
33,Astorga,-23.2325,-51.6656,22.00,78,0,3.60,BR,1618620571
...,...,...,...,...,...,...,...,...,...
509,Carballo,43.2130,-8.6910,8.33,75,0,1.03,ES,1618620780
525,Presidencia Roque Sáenz Peña,-26.7852,-60.4388,22.22,95,0,2.47,AR,1618620786
527,Kargasok,59.0556,80.8572,-7.03,51,0,2.27,RU,1618620787
536,Chadiza,-14.0678,32.4392,14.06,87,0,2.41,ZM,1618620791


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [38]:
hotel_df = ideal_df
hotel_df["Hotel Name"] = ''
hotel_df

<ipython-input-38-febfa1e05abb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ''


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
3,Port Elizabeth,-33.9180,25.5701,16.00,72,0,6.69,ZA,1618620560,
18,Chatrapur,19.3500,84.9833,26.64,73,0,3.39,IN,1618620565,
24,Xianshuigu,38.9850,117.3828,13.00,19,0,7.00,CN,1618620568,
27,Cape Town,-33.9258,18.4232,18.33,77,0,0.51,ZA,1618620395,
33,Astorga,-23.2325,-51.6656,22.00,78,0,3.60,BR,1618620571,
...,...,...,...,...,...,...,...,...,...,...
509,Carballo,43.2130,-8.6910,8.33,75,0,1.03,ES,1618620780,
525,Presidencia Roque Sáenz Peña,-26.7852,-60.4388,22.22,95,0,2.47,AR,1618620786,
527,Kargasok,59.0556,80.8572,-7.03,51,0,2.27,RU,1618620787,
536,Chadiza,-14.0678,32.4392,14.06,87,0,2.41,ZM,1618620791,


In [40]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get hotel based on lat/long
    lat = row["Lat"]
    long = row["Lng"]

    # add keyword to params dict
    params['location'] = f'{lat},{long}'

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Closest hotel is 39 On Nile Guest House.
------------


C:\Users\xiao_\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Closest hotel is Circuit House.
------------
Closest hotel is Guangchuan Hotel.
------------
Closest hotel is Southern Sun Waterfront Cape Town.
------------
Closest hotel is Rancho Jads.
------------
Closest hotel is Nuevo Hotel Plaza.
------------
Closest hotel is Tourist House - Sablino.
------------
Closest hotel is Prival.
------------
Closest hotel is Misty Waves Boutique Hotel.
------------
Closest hotel is ibis Styles Karratha.
------------
Closest hotel is Gran Hotel Mar del Plata.
------------
Closest hotel is Hotel Kap Farvel.
------------
Closest hotel is Mahavir Farm House.
------------
Closest hotel is Royal Exchange Hotel.
------------
Closest hotel is Hospitality Carnarvon.
------------
Closest hotel is Protea Hotel by Marriott Knysna Quays.
------------
Closest hotel is Tortuga B&B.
------------
Closest hotel is Oyster Bay House Rental.
------------
Closest hotel is Complejo de turismo Rural A Toca.
------------
Missing field/result... skipping.
------------
Closest ho

In [41]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
3,Port Elizabeth,-33.9180,25.5701,16.00,72,0,6.69,ZA,1618620560,39 On Nile Guest House
18,Chatrapur,19.3500,84.9833,26.64,73,0,3.39,IN,1618620565,Circuit House
24,Xianshuigu,38.9850,117.3828,13.00,19,0,7.00,CN,1618620568,Guangchuan Hotel
27,Cape Town,-33.9258,18.4232,18.33,77,0,0.51,ZA,1618620395,Southern Sun Waterfront Cape Town
33,Astorga,-23.2325,-51.6656,22.00,78,0,3.60,BR,1618620571,Rancho Jads
...,...,...,...,...,...,...,...,...,...,...
509,Carballo,43.2130,-8.6910,8.33,75,0,1.03,ES,1618620780,Pazo do Souto
525,Presidencia Roque Sáenz Peña,-26.7852,-60.4388,22.22,95,0,2.47,AR,1618620786,Hotel Gualok
527,Kargasok,59.0556,80.8572,-7.03,51,0,2.27,RU,1618620787,Hotel Kargasok
536,Chadiza,-14.0678,32.4392,14.06,87,0,2.41,ZM,1618620791,Sankhani Guest House


In [42]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [43]:
# Add marker layer ontop of heat map

marker = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))